In [1]:
import pandas as pd
import os
from collections import defaultdict
import numpy as np
import re

In [2]:
def def_value(): 
    return 0


def calc_interval(bottom, top):
    
    notes = {
        "C":0,
        "D":1,
        "E":2,
        "F":3,
        "G":4,
        "A":5,
        "B":6
    }

    clockVals = {
        "C": 0,
        "D": 2,
        "E": 4,
        "F": 5,
        "G": 7,
        "A": 9,
        "B": 11
    }

    intervalToClock = [0, 2, 4, 5, 7, 9, 11]

    accidentals = defaultdict(def_value)
    accidentals["--"] = -2
    accidentals["-"] = -1
    accidentals["#"] =  1
    accidentals["##"] = 2    

    bottomNote = re.match("[A-G]", bottom).group()
    topNote = re.match("([A-G])", top).group()
    bAccMatch = re.search("[-#]+", bottom)
    tAccMatch = re.search("[-#]+", top)
    

    bottomAcc = bAccMatch.group() if bAccMatch else " "
    topAcc = tAccMatch.group() if tAccMatch else " "

    bottomInd = notes[bottomNote]
    topInd = notes[topNote]
    noteInterval = (topInd - bottomInd) % 7 + 1

    bottomClock = clockVals[bottomNote] + accidentals[bottomAcc];
    topClock = clockVals[topNote] + accidentals[topAcc];
    clockValue = topClock - bottomClock;

    if clockValue < 0:
        clockValue += 12

    noteToClock = intervalToClock[noteInterval - 1];
    diff = clockValue - noteToClock;

    #The most we'd expect to have to alter an interval is by 6 semitones;
    # otherwise, something is wrong.
    if abs(diff) > 6:
        if diff > 0:
            diff -= 12
        else:
            diff += 12

    ss = "";
    if diff > 0:
        ss = "#"*diff
    elif diff < 0:
        ss = "b" * (-1*diff)
        
    return ss + str(noteInterval)

In [3]:
calc_interval("C-", "G")

'#5'

In [4]:
os.chdir("../Humdrum/HarmonicTranscriptions/Edited/")

In [5]:
# os.chdir("../Humdrum/CompleteTranscriptions/Originals/")
files = os.listdir()

In [6]:
files

['WilsonPickett_ImInLove_1967.hum',
 'BobSeger_AmericanStorm_1986.hum',
 'PeggyLee_IsThatAllThereIs_1969.hum',
 'Chicago_25Or6To4_1970.hum',
 'MilliVanilli_GirlImGonnaMissYou_1989.hum',
 'KoolAndTheGang_JungleBoogie_1974.hum',
 'ArethaFranklin_INeverLovedAMan_1967.hum',
 'JohnDenver_AnniesSong_1974.hum',
 'MarvinGaye_MercyMercyMe_1971.hum',
 'JimmyRuffin_IvePassedThisWayBefore_1967.hum',
 'ToddRundgren_CanWeStillBeFriends_1978.hum',
 'Chicago_MakeMeSmile_1970.hum',
 'Shannon_LetTheMusicPlay_1984.hum',
 'GladysKnightAndThePips_IfIWereYourWoman_1971.hum',
 'RitaCoolidge_WereAllAlone_1977.hum',
 'Commodores_Lady_1981.hum',
 'CliffRichard_DevilWoman_1976.hum',
 'BruceSpringsteen_ImGoinDown_1985.hum',
 'UB40_TheWayYouDoTheThingsYouDo_1990.hum',
 'KennyRogers_Lucille_1977.hum',
 'WilsonPhillips_HoldOn_1990.hum',
 'RitaCoolidge_HigherAndHigher_1977.hum',
 'JacksonBrowne_RedneckFriend_1973.hum',
 'JoanBaez_ThereButForFortune_1965.hum',
 'Blondie_IslandOfLostSouls_1982.hum',
 'RDeanTaylor_India

In [7]:
os.getcwd()

'/Users/johnmcnamara/Documents/GitHub/CoCoPops-Billboard-RollingStone-Combined/Billboard/Humdrum/HarmonicTranscriptions/Edited'

In [14]:
def createHarte(humFile):

    df = pd.read_csv(humFile, sep = "\t")
    chordsym = df["**chordsym"].copy()
    harmony = df["**harmony"].copy()
    df["**harte"] = chordsym
    matches = chordsym.str.extractall(r'([A-G][-#]?)').dropna()
    match_indicies = list(set([matches.index[i][0] for i in range(len(matches.index))]))
    match_indicies.sort()
    proto_harte = chordsym.str.extract("([A-G][-#]?:?\w*\(?[^A-G ]*\)?)").dropna()
    for i in match_indicies:
        notes = matches.loc[i][0]
        if len(notes) > 1:
            bass_interval = calc_interval(notes[0], notes[1])
            bass_interval = "" if bass_interval == "1" else "/" + bass_interval
            proto_harte.loc[i][0] =  proto_harte.loc[i][0] + bass_interval

    for i in proto_harte.index[1:]:
        chordsym.loc[i] = proto_harte.loc[i][0]

    df["**harte"] = chordsym
#     if export:
#         df.to_csv("../HarmRhythm/"+humFile, sep="\t")
    return df

In [13]:
createHarte(files[0])

KeyError: '**chordsym'

In [12]:
for file in files:
    createHarmRhythm(file, True)

In [10]:
def createHarmRhythm(humFile, export = False):
    df = pd.read_csv(humFile, sep = "\t")
    if "**harmony" not in df:
        return
    harmony = df["**harmony"].copy()
    df["**harmrhythm"] = harmony
    matches = harmony.str.extract(r'(^\d+\.*)').dropna()
    for i in matches.index:
        df["**harmrhythm"][i] = matches[0].loc[i]
    if export:
        df.to_csv("../HarmRhythm/"+humFile, sep="\t")
    return df

In [11]:
x=createHarmRhythm(files[0])["**harmrhythm"]
for i in x:
    print(i)

*M4/4
*E:
1
=1
*>A_intro
2
4
4
=2
2
2
=3
2
4
4
=4
2
2
=5
2
4
4
=6
2
2
=7
2
4
4
=8
2
2
=9
*>A2_verse
2
4
4
=10
2
2
=11
2
4
4
=12
2
2
=13
2
4
4
=14
2
2
=15
2
4
4
=16
2.
4
=17
*>B_prechorus
1
=18
1
=19
1
=20
1
=21
*>C_chorus
2
4
4
=22
1
=23
2
4
4
=24
1
=25
*>A2_verse
2
4
4
=26
2
2
=27
2
4
4
=28
2
2
=29
2
4
4
=30
2
2
=31
2
4
4
=32
2.
4
=33
*>B_prechorus
1
=34
1
=35
1
=36
1
=37
*>C_chorus
2
4
4
=38
1
=39
2
4
4
=40
2
4
4
=41
*>B_solo
1
=42
1
=43
1
=44
1
=45
*>A_chorus
2
4
4
=46
2
2
=47
2
4
4
=48
2
2
=49
2
4
4
=50
2
2
=51
2
4
4
=52
2
2
=53
*>_fadeout
2
4
4
=54
2
2
=55
2
4
4
=56
2
2
1
1
*-
!!!OTL: I'm In Love
!!!COC: Wilson Pickett
!!!BillboardChartDate: 1967/12/02
!!!BillboardPeak: 45
!!!BillboardWeeksOnChart: 10
!!!SampleTargetRank: 83
!!!SampleActualRank: 82
!!!SampleID#: 1221
!!!RDF**harmony: **recip + pitch class of root + harmonic quality
!!!RDF**chordsym: original McGill chord annotation + (implied pitch class set, with bass pitch in first position)
!!!RDF**phrase: newline = linebreak i

In [47]:
for i in matches.index:
    df["**harmrhythm"][i] = matches[0].loc[i]

In [48]:
for x in df["**harmrhythm"]:
    print(x)

*M4/4
*
*
*a:
*
*
1
=1
*>Letter>C
*>Label>Intro
1
=2
1
=3
1
=4
2
2
=5
1
=6
1
=7
1
=8
2
2
=9
1
=10
1
=11
1
=12
2
2
=13
1
=14
1
=15
1
=16
2
2
=17
*>Letter>A
*>Label>Verse
1
.
.
.
=18
1
.
.
.
.
=19
1
.
.
=20
2
2
=21
1
.
.
.
=22
1
.
.
.
.
=23
1
.
.
.
=24
2
2
=25
1
.
.
.
=26
1
.
.
.
.
=27
1
.
=28
2
2
=29
1
.
.
.
=30
1
.
.
.
.
=31
1
.
.
.
.
=32
2
2
=33
*>Letter>B
*>Label>Chorus
1
.
.
.
=34
1
.
.
.
.
=35
1
=36
1
.
=37
1
.
.
.
=38
1
.
.
.
.
=39
1
=40
1
.
=41
*>Letter>K
*>Label>Interlude
1
=42
1
=43
1
=44
2
2
=45
1
=46
1
=47
1
=48
2
2
=49
*>Letter>A
*>Label>Verse
1
.
.
.
=50
1
.
.
.
.
=51
1
.
.
=52
2
2
=53
1
.
.
.
=54
1
.
.
.
.
=55
1
.
=56
2
2
=57
1
.
.
.
=58
1
.
.
.
.
=59
1
.
=60
2
2
=61
1
.
.
.
=62
1
.
.
.
.
=63
1
.
.
.
.
=64
2
2
=65
*>Letter>B
*>Label>Chorus
1
.
.
.
=66
1
.
.
.
.
=67
1
=68
1
.
=69
1
.
.
.
=70
1
.
.
.
.
=71
1
.
=72
1
=73
*>Letter>D
*>Label>Instrumental
1
.
.
.
=74
1
=75
1
=76
2
2
=77
1
=78
1
=79
1
=80
2
2
=81
1
=82
1
=83
1
=84
2
2
=85
1
=86
1
=87
1
=88
2
2
=89
1
=90
1
=91
1
=